## Question 3

In Question 3, your objective is to find the best available product on Daraz, specifically the Redmi Note 12 smartphone. You will scrape important information from the products listed on the first five pages, which includes the Product Title, Rating, Free Shipping Status, Daraz Mall status, Seller Rating and the Ship on Time score.

To find the best available product, apply the following scoring criteria to each product:

<b>1) </b> The price will be the base score.<br>
<b>2) </b> Award an additional 5,000 points to products with the "Daraz Mall" status.<br>
<b>3) </b> Grant an extra 1,000 points to products with free shipping.<br>
<b>4) </b> Add 10000*((0.4 * (Product Rating / 5)) + (0.4 * (Seller Rating / 100)) + (0.2 * (Shipping Performance / 100)) to the score <br>

In [1]:
import pandas as pd
import time
import numpy as np

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import InvalidSelectorException

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import nltk

In [ ]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
daraz_url = "https://www.daraz.pk/"    
driver.get(daraz_url)

In [ ]:
search_bar = driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div[1]/div/div/div[2]/div/div[2]/form/div/div[1]/input[1]')
search_bar.send_keys('Redmi Note 12',Keys.RETURN)
time.sleep(5)

In [ ]:
data = []

for x in range(5):
    boxes = driver.find_elements(By.CLASS_NAME, "gridItem--Yd0sa")

    for box in boxes:
        details = box.text.split('\n')
        name = details[0]
        price = details[1]
        free_shipping = 'Free Shipping' in details

        product_link = box.find_element(By.TAG_NAME, "a")
        product_link = product_link.get_attribute("href")

        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])
        driver.get(product_link)

        try:
            shiping_time = driver.find_element(By.XPATH, "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[2]").text
            shiping_time = shiping_time.split('\n')
            shiping_time = shiping_time[1]
            shiping_time

            row =driver.find_element(By.XPATH,"/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[1]").text
            seller_rating = row.split('\n')
            seller_rating = seller_rating[1]
            if '%' not in seller_rating:
                seller_rating = '0%'

        except:
            driver.execute_script('window.scrollTo(0, 500) ')
            shiping_time = driver.find_element(By.XPATH, "/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[2]").text
            shiping_time = shiping_time.split('\n')
            shiping_time = shiping_time[0]

        try:
            # driver.execute_script('window.scrollTo(0,500) ')
            time.sleep(1)
            rating = driver.find_element(By.XPATH, "/html/body/div[4]/div/div[9]/div[1]/div[2]/div/div/div/div[1]/div[2]")
        except NoSuchElementException:
            # driver.execute_script('window.scrollTo(0, 1450) ')
            time.sleep(1)
            rating = driver.find_element(By.XPATH, "/html/body/div[4]/div/div[9]/div[1]/div[2]/div/div/div/div[1]/div[2]")
        
        rating = rating.text.split('\n')
        rating = rating[0].split('/')
        rating = rating[0]

        product_rating = rating

        h = driver.find_element(By.XPATH,"/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[3]/div/div")
        Daraz_mall = False
        try:
            i=h.find_element(By.TAG_NAME,"img")
            Daraz_mall = True
        except NoSuchElementException:
            Daraz_mall = False

    
        
        data.append({"Product Title": name, "Price": price, "Rating": product_rating, "Free Shipping Status": free_shipping,
                     "Daraz Mall Status": Daraz_mall, "Seller Rating": seller_rating, "Ship on Time Score": shiping_time,"Link":product_link})
        
        
        driver.close()

        driver.switch_to.window(driver.window_handles[0])

    try:
        driver.execute_script('window.scrollTo(0,4200) ')
        time.sleep(1)
        button = driver.find_element(By.XPATH, "/html/body/div[3]/div/div[3]/div/div/div[1]/div[3]/div/div/ul/li[9]/a")
        button.click()
        time.sleep(2)
    except NoSuchElementException:
        break

daraz_df = pd.DataFrame(data)


In [ ]:
daraz_df.to_csv("Daraz_Data.csv")

In [4]:
dff = pd.read_csv('Daraz_Data.csv',index_col="Unnamed: 0")
dff.head()

,Product Title,Price,Rating,Free Shipping Status,Daraz Mall Status,Seller Rating,Ship on Time Score,Link
0,Xiaomi Redmi Note 12 || 8GB Ram 128GB Rom || 5...,"Rs. 51,499",4.7,True,False,91%,82%,https://www.daraz.pk/products/12-8gb-128gb-500...
1,"Redmi Note 12 - 6.67"" Inch Display - 8GB RAM -...","Rs. 51,299",4.1,True,False,85%,94%,https://www.daraz.pk/products/redmi-note-12-66...
2,Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB F...,"Rs. 88,840",5.0,True,False,93%,100%,https://www.daraz.pk/products/12-8-gb-256-gb-1...
3,Redmi Note 12 Pro - 8GB Ram 256GB Rom - 5000mA...,"Rs. 87,999",4.5,True,False,91%,82%,https://www.daraz.pk/products/12-8gb-256gb-500...
4,Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB F...,"Rs. 88,880",5.0,True,False,96%,96%,https://www.daraz.pk/products/12-8-gb-256-gb-1...


In [5]:
dff['Price'] = pd.to_numeric(dff['Price'].str.replace("Rs","").str.replace(".","").str.replace(",",""))
dff['Seller Rating'] = pd.to_numeric(dff['Seller Rating'].str.replace('%',''))
dff['Ship on Time Score'] = pd.to_numeric(dff['Ship on Time Score'].str.replace('%',''))
dff.head()

,Product Title,Price,Rating,Free Shipping Status,Daraz Mall Status,Seller Rating,Ship on Time Score,Link
0,Xiaomi Redmi Note 12 || 8GB Ram 128GB Rom || 5...,51499,4.7,True,False,91,82,https://www.daraz.pk/products/12-8gb-128gb-500...
1,"Redmi Note 12 - 6.67"" Inch Display - 8GB RAM -...",51299,4.1,True,False,85,94,https://www.daraz.pk/products/redmi-note-12-66...
2,Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB F...,88840,5.0,True,False,93,100,https://www.daraz.pk/products/12-8-gb-256-gb-1...
3,Redmi Note 12 Pro - 8GB Ram 256GB Rom - 5000mA...,87999,4.5,True,False,91,82,https://www.daraz.pk/products/12-8gb-256gb-500...
4,Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB F...,88880,5.0,True,False,96,96,https://www.daraz.pk/products/12-8-gb-256-gb-1...


In [6]:

# Create a new column for the scores
dff['Score'] = dff['Price']

# # Add 5,000 points for Daraz Mall status
dff['Score'] += dff['Daraz Mall Status'] * 5000

# # Add 1,000 points for free shipping
dff['Score'] += dff['Free Shipping Status'] * 1000


# # Calculate the score based on the provided formula
dff['Score'] += 10000 * ((0.4 * (dff['Rating'] / 5)) + (0.4 * (dff['Seller Rating'] / 100)) + (0.2 * (dff['Ship on Time Score'] / 100)))

# # Find the product with the highest score
best_product = dff.loc[dff['Score'].idxmax()]

# Print or use the information of the best product
print("Best Product:",best_product['Product Title'])

product_link = best_product['Link']

Best Product: Redmi Note 12 Pro (8GB-256GB) PTA Approved With Official 1 Year Warranty


# Question 4

Having successfully identified the best product, proceed to its product page on Daraz. Your task is to scrape the product's reviews and perform sentiment analysis on these reviews. To conduct sentiment analysis, you can utilize the NLTK (Natural Language Toolkit) library, which will help you determine the sentiment expressed in the text of the reviews.

In [ ]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))    
driver.get(product_link)

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

review = []
analysis = []

pages_to_scrap = 8

for i in range(pages_to_scrap):
    
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, 500);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(500, 800);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(800, 1000);")
    time.sleep(2)
    driver.execute_script("window.scrollTo(1000, 1300);")
    time.sleep(2)

    review_box = driver.find_elements(By.CLASS_NAME,'review-item')

    for x in review_box:
        reviews = x.find_element(By.CLASS_NAME, 'review-content-sl')
        if reviews.text != "":
        
            # Perform sentiment analysis
            sentiment_scores = analyzer.polarity_scores(reviews.text)

            # Determine sentiment (positive, negative, or neutral)
            if sentiment_scores['compound'] >= 0.05:
                sentiment = 'Positive'
            elif sentiment_scores['compound'] <= -0.05:
                sentiment = 'Negative'
            else:
                sentiment = 'Neutral'

            analysis.append({'Review': reviews.text, 'Sentiment': sentiment, 'Polarity Scores': sentiment_scores})
            
    try:
        button = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[3]/div[4]/ul/li[9]/button')
        button.click()
    except NoSuchElementException:
        try:
            button = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[3]/div[4]/ul/li[10]/button')
            button.click()
        except NoSuchElementException:
            button = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[3]/div[4]/ul/li[11]/button')
            button.click()

In [ ]:
reviewss = pd.DataFrame(analysis)
reviewss

In [ ]:
reviewss["Sentiment"].value_counts()